## Test Chunks 

In [13]:
import os 
import re
import time
import copy
import fitz
import numpy as np
import pinecone as pc
from tqdm.auto import tqdm
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.vectorstores import  Pinecone ,FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()

class CustomTextSplitter:

    def __init__(self, chunk_size, chunk_overlap):
        self.keep_separator= False
        self.strip_whitespace=True
        self.is_separator_regex=False
        self.add_start_index=False
        self.length_function = len
        self.chunk_size=chunk_size
        self.chunk_overlap = chunk_overlap
        self.separators=["\n\n", "\n", " ", ""]

    def _split_text_with_regex(self,text: str, separator: str):
        # Now that we have the separator, split the text
        if separator:
            if self.keep_separator:
                # The parentheses in the pattern keep the delimiters in the result.
                _splits = re.split(f"({separator})", text)
                splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
                if len(_splits) % 2 == 0:
                    splits += _splits[-1:]
                splits = [_splits[0]] + splits
            else:
                splits = re.split(separator, text)
        else:
            splits = list(text)
        return [s for s in splits if s != ""]
    
    def _join_docs(self, docs, separator:str,):
        text = separator.join(docs)
        if self.strip_whitespace:
            text = text.strip()
        if text == "":
            return None
        else:
            return text
        

    def _merge_splits(self, splits, separator: str):
        # We now want to combine these smaller pieces into medium size
        # chunks to send to the LLM.
        separator_len = self.length_function(separator)

        docs = []
        current_doc= []
        total = 0
        for d in splits:
            _len = self.length_function(d)
            if (
                total + _len + (separator_len if len(current_doc) > 0 else 0)
                > self.chunk_size
            ):
                if total > self.chunk_size:
                    print(
                        f"Created a chunk of size {total}, "
                        f"which is longer than the specified {self.chunk_size}"
                    )
                if len(current_doc) > 0:
                    doc = self._join_docs(current_doc, separator)
                    if doc is not None:
                        docs.append(doc)
                    # Keep on popping if:
                    # - we have a larger chunk than in the chunk overlap
                    # - or if we still have any chunks and the length is long
                    while total > self.chunk_overlap or (
                        total + _len + (separator_len if len(current_doc) > 0 else 0)
                        > self.chunk_size
                        and total > 0
                    ):
                        total -= self.length_function(current_doc[0]) + (
                            separator_len if len(current_doc) > 1 else 0
                        )
                        current_doc = current_doc[1:]
            current_doc.append(d)
            total += _len + (separator_len if len(current_doc) > 1 else 0)
        doc = self._join_docs(current_doc, separator)
        if doc is not None:
            docs.append(doc)
        return docs
    
    def _split_text(self,text: str,):
        """Split incoming text and return chunks."""
        final_chunks = []
        # Get appropriate separator to use
        separator = self.separators[-1]
        new_separators = []
        for i, _s in enumerate(self.separators):
            _separator = _s if self.is_separator_regex else re.escape(_s)
            if _s == "":
                separator = _s
                break
            if re.search(_separator, text):
                separator = _s
                new_separators = self.separators[i + 1 :]
                break

        _separator = separator if self.is_separator_regex else re.escape(separator)
        splits = self._split_text_with_regex(text, _separator)

        # Now go merging things, recursively splitting longer texts.
        _good_splits = []
        _separator = "" if self.keep_separator else separator
        for s in splits:
            if self.length_function(s) < self.chunk_size:
                _good_splits.append(s)
            else:
                if _good_splits:
                    merged_text = self._merge_splits(_good_splits, _separator)
                    final_chunks.extend(merged_text)
                    _good_splits = []
                if not new_separators:
                    final_chunks.append(s)
                else:
                    other_info = self._split_text(s, new_separators)
                    final_chunks.extend(other_info)
        if _good_splits:
            merged_text = self._merge_splits(_good_splits, _separator)
            final_chunks.extend(merged_text)
        return final_chunks
    

    def create_documents(self, texts, pdf_name):
        """Create documents from a list of texts."""
        documents = []
        for i, text in enumerate(texts):
            index = -1
            for chunk in self._split_text(text):
                if self.add_start_index:
                    index = text.find(chunk, index + 1)
                new_doc = Document(page_content=chunk, metadata={})
                new_doc.metadata['source'] = pdf_name
                documents.append(new_doc)
        return documents
    
class PdfTextExtractor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path 
        self.start_page = 1
        self.end_page = None

    def _preprocess(self,text):
        '''
        preprocess extrcted text from pdf
        1. Replace new line character with whitespace.
        2. Replace redundant whitespace with a single whitespace
        '''
        text = text.replace('\n', ' ')
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\\u[e-f][0-9a-z]{3}',' ', text)
        return text
    
    def _pdf_to_text(self, pdf_filename):
        '''
            convert pdf to a list of words.
        '''
        doc = fitz.open(self.pdf_path)
        total_pages= doc.page_count

        if self.end_page is None:
            self.end_page = total_pages
        text_list=[]

        for i in tqdm(range(self.start_page-1, self.end_page)):
            text= doc.load_page(i).get_text('text')
            text= self._preprocess(text)
            text_list.append(f'[{pdf_filename}, page:{i+1}] '+text+ f' [{pdf_filename}, page:{i+1}]')
        doc.close()
        return text_list
    


class Data:

    def __init__(self, pdf_data_path, vector_db_path):

        self.pdf_data_path = pdf_data_path
        self.vector_db_path = vector_db_path
        self.pinecone_api_key = os.getenv('PINECONE_KEY')
        self.pinecone_env = os.getenv('PINECONE_ENV')
        self.embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        self.openai_embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002',
                                                        openai_api_key=os.getenv('OPENAI_KEY'))
        
    def createPDFVectorDBwithFAISS(self, chunk_size, chunk_overlap):
        document_list=[]
        for pdf_filename in os.listdir(self.pdf_data_path):
            pdf_file_path = os.path.join(self.pdf_data_path,pdf_filename)
            extracted_text_list = PdfTextExtractor(pdf_file_path)._pdf_to_text(pdf_filename)
            merged_text_list = ['.'.join(extracted_text_list)]
            splitter = CustomTextSplitter(chunk_size, chunk_overlap)
            docs  = splitter.create_documents(merged_text_list,pdf_filename)
            document_list.extend(docs)
    
        db = FAISS.from_documents(document_list, self.embedding_model)
        db.save_local(self.vector_db_path)

    def create_top_k_chunk_from_FAISS(self, question,top_k):
        test_idex = FAISS.load_local(self.vector_db_path,self.embedding_model)
        top_k_chunks  = test_idex.similarity_search(question,k=top_k)
        return top_k_chunks
    
    def fetch_FAISS_VectorDB(self):
        test_index = FAISS.load_local(self.vector_db_path,self.embedding_model)
        return test_index
    

    def createPDFVectorDBwithPinecone(self,chunk_size, chunk_overlap):
        document_list=[]
        for pdf_filename in os.listdir(self.pdf_data_path):
            pdf_file_path = os.path.join(self.pdf_data_path,pdf_filename)
            extracted_text_list = PdfTextExtractor(pdf_file_path)._pdf_to_text(pdf_filename)
            merged_text_list = ['.'.join(extracted_text_list)]
            splitter = CustomTextSplitter(chunk_size, chunk_overlap)
            docs  = splitter.create_documents(merged_text_list,pdf_filename)
            document_list.extend(docs)

        embeddings = []
        ids = []
        metadatas = []
        for i in range(len(document_list)):
            if i%5==0:
                time.sleep(5)
            page_content = document_list[i].page_content
            source_pdf = document_list[i].metadata['source'].split('\\')[-1]
            embedded_page_content = self.openai_embedding_model.embed_query(page_content)
            metadata = {
                'source' : source_pdf,
                'page_content' : page_content
            }
            ids.append(str(i))
            embeddings.append(embedded_page_content)
            metadatas.append(metadata)

        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        index.upsert(vectors = zip(ids, embeddings, metadatas))

    def create_top_k_chunk_from_Pinecone(self, question,top_k):
        pc.init(api_key=self.pinecone_api_key, environment=self.pinecone_env)
        index = pc.Index('pdf-index')
        vectorstore = Pinecone( index, self.openai_embedding_model.embed_query, text_key='page_content')
        top_k_chunks = vectorstore.similarity_search(question, k=top_k)
        return top_k_chunks



In [14]:
pdf_data_path = ".\\media"
pdf_vector_embedding_path = ".\\VectorDB"
data_obj = Data(pdf_data_path,pdf_vector_embedding_path)
data_obj.createPDFVectorDBwithFAISS(chunk_size=2000, chunk_overlap=500)

100%|██████████| 4/4 [00:00<00:00, 351.13it/s]


In [15]:
test_question = "Artificial Intelligence in budget?"
result = data_obj.create_top_k_chunk_from_FAISS(test_question, top_k =3)
result

[Document(page_content='key to a nation’s progress. Our government is committed to providing a transparent and accountable administration which works for the betterment and welfare of the common citizen,” said Hon’ble Prime Minister. [budget_2023.pdf, page:17].[budget_2023.pdf, page:18] 14 Mission Karmayogi 58. Under Mission Karmayogi, Centre, States and Union Territories are making and implementing capacity-building plans for civil servants. The government has also launched an integrated online training platform, iGOT Karmayogi, to provide continuous learning opportunities for lakhs of government employees to upgrade their skills and facilitate people-centric approach. 59. For enhancing ease of doing business, more than 39,000 compliances have been reduced and more than 3,400 legal provisions have been decriminalized. For furthering the trust- based governance, we have introduced the Jan Vishwas Bill to amend 42 Central Acts. This Budget proposes a series of measures to unleash the po

In [22]:
pdf_data_path = ".\\media"
pdf_vector_embedding_path = ".\\VectorDB"
data_obj = Data(pdf_data_path,pdf_vector_embedding_path)
test_question = "Artificial Intelligence in budget?"
result = data_obj.create_top_k_chunk_from_FAISS(test_question, top_k=3)
result

[Document(page_content='key to a nation’s progress. Our government is committed to providing a transparent and accountable administration which works for the betterment and welfare of the common citizen,” said Hon’ble Prime Minister. [budget_2023.pdf, page:17].[budget_2023.pdf, page:18] 14 Mission Karmayogi 58. Under Mission Karmayogi, Centre, States and Union Territories are making and implementing capacity-building plans for civil servants. The government has also launched an integrated online training platform, iGOT Karmayogi, to provide continuous learning opportunities for lakhs of government employees to upgrade their skills and facilitate people-centric approach. 59. For enhancing ease of doing business, more than 39,000 compliances have been reduced and more than 3,400 legal provisions have been decriminalized. For furthering the trust- based governance, we have introduced the Jan Vishwas Bill to amend 42 Central Acts. This Budget proposes a series of measures to unleash the po

## Test - Model

In [23]:
import os 
import re
import openai
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

load_dotenv()

class Model:
    def __init__(self):
        self.openai_api_key = os.getenv("OPENAI_KEY")
        self.model = 'text-davinci-003'
        self.llm = ChatOpenAI( openai_api_key = os.getenv("OPENAI_KEY"))

    def createQuestionPrompt(self, top_k_chunks):
        prompt= ""
        prompt += 'Context:\n\n'
        for i in range(len(top_k_chunks)):
            page_content = top_k_chunks[i].page_content.replace('\n', ' ')
            page_content = re.sub('\s+', ' ', page_content)
            prompt += page_content +'\n\n'
        prompt += '''\nInstructions: Compose a comprehensive reply to the query asked by the user from the context provided.
        Cite each reference using [pdfname.pdf , page : number] notation (every result has this number at the beginning and end).
        Citation should be done at the end of each sentence. If the search results mention multiple subjects
        with the same name, create separate answers for each. Only include information found in the results and
        don't add any additional information. Make sure the answer is correct and don't output false content.
        If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'
        Directly start the answer.\n'''.replace('\\n',' ')
    
        return prompt
    
    def createQuestionPromptTemplate(self, prompt):
        prompt_template_llmchain = ChatPromptTemplate.from_messages([
                    SystemMessage(content=f"You are a QnA Chatbot whose job is to greet the user politely and asnwer to the question they ask."+prompt), 
                    MessagesPlaceholder(variable_name="chat_history"),         # Where the memory will be stored.
                    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
                 ])
        
        return prompt_template_llmchain
    
    def generateAnswer(self, prompt):
        openai.api_key = self.openai_api_key
        completions = openai.Completion.create(
            engine=self.model,
            prompt=prompt,
            max_tokens=1024,
            temperature=0,
        )
        answer = completions.choices[0]['text']
        return answer
    
    def generateAnswerwithMemory(self,question, prompt_template, chat_history):
        memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=3)
        llm = ChatOpenAI(openai_api_key = os.getenv("OPENAI_KEY"),model='gpt-3.5-turbo')
        llm_chain = LLMChain(
            llm=llm,
            prompt=prompt_template,
            verbose=True,
            memory=memory,
        )

        llm_chain_response = llm_chain.predict(human_input = question)
        interaction = 'human:'+question+'\nchatbot:'+llm_chain_response
        chat_history.append(interaction)
        return llm_chain_response
    
    

In [24]:
model_obj = Model()
prompt = model_obj.createQuestionPrompt(result)
prompt_template = model_obj.createQuestionPromptTemplate(prompt)
response = model_obj.generateAnswerwithMemory(test_question, prompt_template,chat_history=[])
response




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a QnA Chatbot whose job is to greet the user politely and asnwer to the question they ask.Context:

key to a nation’s progress. Our government is committed to providing a transparent and accountable administration which works for the betterment and welfare of the common citizen,” said Hon’ble Prime Minister. [budget_2023.pdf, page:17].[budget_2023.pdf, page:18] 14 Mission Karmayogi 58. Under Mission Karmayogi, Centre, States and Union Territories are making and implementing capacity-building plans for civil servants. The government has also launched an integrated online training platform, iGOT Karmayogi, to provide continuous learning opportunities for lakhs of government employees to upgrade their skills and facilitate people-centric approach. 59. For enhancing ease of doing business, more than 39,000 compliances have been reduced and more than 3,400 legal provisions have been decriminalized. For furthering th

'Three centers of excellence for Artificial Intelligence will be set up in top educational institutions as part of the budget. These centers will partner with leading industry players to conduct interdisciplinary research, develop cutting-edge applications, and provide scalable problem solutions in the areas of agriculture, health, and sustainable cities. The aim is to build an effective AI ecosystem and nurture quality human resources in this field [budget_2023.pdf, page:18].'

## Test - SQL

In [25]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv
import mysql.connector as connection 
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')
openai.api_key = os.getenv('OPENAI_KEY')

class SQLQuerywithLangchain:
    def __init__(self):
        self.DB_USERNAME = os.getenv('DB_USERNAME')
        self.DB_PASSWORD = os.getenv('DB_PASSWORD')
        self.DB_HOST = os.getenv('DB_HOST')
        self.DB_PORT = os.getenv('DB_PORT')
        self.DB_NAME = os.getenv('DB_NAME')

    def createDBConnectionString(self):
        db_user = self.DB_USERNAME
        db_Password  = self.DB_PASSWORD
        db_host = self.DB_HOST + self.DB_PORT
        db_name = self.DB_NAME
        connectionString = f"mysql+pymysql://{db_user}:{db_Password}@{db_host}/{db_name}"
        return connectionString
    
    def getSQLSchema(self):
            ''''
                Extracting the schema info from the MySQL database and passing the schema 
                information to the prompt.
            '''
            sql_query = f"""  
            SELECT C.TABLE_NAME, C.COLUMN_NAME, C.DATA_TYPE, T.TABLE_TYPE, T.TABLE_SCHEMA  
            FROM INFORMATION_SCHEMA.COLUMNS C  
            JOIN INFORMATION_SCHEMA.TABLES T ON C.TABLE_NAME = T.TABLE_NAME AND C.TABLE_SCHEMA = T.TABLE_SCHEMA  
            WHERE T.TABLE_SCHEMA = '{self.DB_NAME}' 
            """ 
            mysql_connection_string = self.createDBConnectionString()
            result = pd.read_sql_query(sql_query, mysql_connection_string)
            df = result.infer_objects()
            output=[]
            current_table = ''  
            columns = []  
            for index, row in df.iterrows():
                table_name = f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}"  
                column_name = row['COLUMN_NAME']  
                data_type = row['DATA_TYPE']  
                if " " in table_name:
                    table_name= f"[{table_name}]" 
                column_name = row['COLUMN_NAME']  
                if " " in column_name:
                    column_name= f"[{column_name}]" 
                    # If the table name has changed, output the previous table's information  
                if current_table != table_name and current_table != '':  
                    output.append(f"table: {current_table}, columns: {', '.join(columns)}")  
                    columns = []  
                
                # Add the current column information to the list of columns for the current table  
                columns.append(f"{column_name} {data_type}")  
                
                # Update the current table name  
                current_table = table_name  

            # Output the last table's information  
            output.append(f"table: {current_table}, columns: {', '.join(columns)}")
            output = "\n ".join(output)

            return output   

    def createAgentExecutor(self, openAI_model_name="gpt-3.5-turbo"):
        '''
            Instantiating Langchain agent to query SQL Database.
            Using SQLDatabaseToolkit from Langchain.
        '''
        mysql_connection_string = self.createDBConnectionString()
        llm = ChatOpenAI(model_name= openAI_model_name )
        db = SQLDatabase.from_uri(mysql_connection_string)
        toolkit = SQLDatabaseToolkit(db=db, llm =llm)
        agent_executor = create_sql_agent(
                                        llm=llm,
                                        toolkit=toolkit,
                                        verbose=True,
                                        return_intermediate_steps=False,
                                        handle_parsing_errors=True)
        return agent_executor

    def fetchQueryResult(self,question):
        '''
            Using langchain's SQL tool to fetch answer to the user's query.
        '''
        db_agent = self.createAgentExecutor()
        schema_info = self.getSQLSchema()
        prompt = f'''You are a professional SQL Data Analyst whose job is to fetch results from the SQL database.\
            The SQL Table schema is as follows {schema_info}.\
            The question will be asked in # delimiter. If you are not able to find the answer write "Found Nothing" in response.\
            Do not write anything out of context or on your own.\
            If the SQL query returns multiple rows then summarize them and provide response using bullet points.For duplicate response after the SQL query consider any one of the result to parse into LLM.\
            Question : # {question} #'''
        db_agent.return_intermediate_steps=True
        agent_response = db_agent(prompt)
        output = agent_response['output']
        # query = agent_response['intermediate_steps'][-1][0].log.split('\n')[-1].split('Action Input:')[-1].strip().strip('"')
        return output 
    

class SQLQuerywithFunctionCalling(SQLQuerywithLangchain):
    def __init__(self):
        super().__init__()

    def getMYSQLConnectionObject(self):
        db_user = self.DB_USERNAME
        db_password  = self.DB_PASSWORD
        db_host = self.DB_HOST
        db_name = self.DB_NAME
        conn = connection.connect(host=db_host,user=db_user,password=db_password,
                                        database=db_name, use_pure=True) 
        if conn.is_connected():
            return conn
        else:
            return "Database connection can't be established"
    
    def defineFunction(self):
        database_schema_string = self.getSQLSchema()
        function = [
            {
                "name": "ask_database",
                "description": "Use this function to answer user questions about product. Output should be a fully formed SQL query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": f"""
                                    SQL query extracting info to answer the user's question.
                                    SQL should be written using this database schema:
                                    {database_schema_string}
                                    The query should be returned in plain text, not in JSON.
                                    Do not use new lines chatacthers inside the query.
                                    """,
                        }
                    },
                    "required": ["query"],
                },
            }
        ]

        return function
    
    def ask_database(self,query):
        """Function to query MySQL database with a provided SQL query."""
        try:
            conn = self.getMYSQLConnectionObject()
            cursor=conn.cursor()   
            cursor.execute(query)  
            results = str(cursor.fetchall())
            conn.close()
        except Exception as e:
            results = f"query failed with error: {e}"
        return results
    
    def execute_function_call(self,message):
        if message["function_call"]["name"] == "ask_database":
            query = eval(message["function_call"]["arguments"])["query"]
            results = self.ask_database(query)
        else:
            results = f"Error: function {message['function_call']['name']} does not exist"
        return results
    
    def openai_functions_chain(self,query):
        messages = []
        messages.append({"role": "system", "content": F"Answer user questions by generating SQL queries against the {self.DB_NAME} Database."})
        messages.append({"role": "user", "content": query})
        while True:
            assistant_message = openai.ChatCompletion.create(
                temperature=0,
                model="gpt-3.5-turbo-0613",
                messages=messages,
                functions=self.defineFunction(),
                function_call="auto",
            )["choices"][0]["message"]
            messages.append(assistant_message)

            if assistant_message.get("function_call"):
                print("Executing function: ", assistant_message["function_call"])
                results = self.execute_function_call(assistant_message)
                messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
            else:
                break

        return assistant_message['content']

In [7]:
question = "what is the count of mobile device type?"
obj = SQLQuerywithFunctionCalling()
res = obj.openai_functions_chain(question)
print(res)

Executing function:  {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT COUNT(*) FROM mavenfuzzyfactory.website_sessions WHERE device_type = 'mobile'\"\n}"
}
The count of mobile device type is 145,844.


## Test- Transcript

In [20]:
import whisper
import os, glob

def speech2Text(recorded_audio_path, transcripted_audio_path):
    try:
        recordings_dir = os.path.join(recorded_audio_path,'*')
        model = whisper.load_model("base")

        # get most recent wav recording in the recordings directory
        files = sorted(glob.iglob(recordings_dir), key=os.path.getctime, reverse=True)
        latest_recording = files[0]
        print("latest_recording : ", latest_recording)
        latest_recording_filename = latest_recording.split('\\')[-1]
        print("latest_recording_filename : ", latest_recording_filename)

        if os.path.exists(latest_recording):
            audio = whisper.load_audio(latest_recording)
            audio = whisper.pad_or_trim(audio)
            mel = whisper.log_mel_spectrogram(audio).to(model.device)
            options = whisper.DecodingOptions(language= 'en', fp16=False)

            result = whisper.decode(model, mel, options)

            if result.no_speech_prob < 0.5:
                print(result.text)
                textfiles = os.listdir(transcripted_audio_path)
                if len(textfiles)!=0:
                    os.remove(os.path.join(transcripted_audio_path,textfiles[0]))

                transcription_filename = os.path.join(transcripted_audio_path,'input_audio_transcription.txt')#'Stored Data/Input/Transcription/input_audio_transcription.txt'
                with open(transcription_filename, 'a') as f:
                    f.write(result.text)

                return result.text
            
            else:
                alternate_text = "The input audio contains no word"
                textfiles = os.listdir(transcripted_audio_path)
                if len(textfiles)!=0:
                    os.remove(os.path.join(transcripted_audio_path,textfiles[0]))

                transcription_filename = os.path.join(transcripted_audio_path,'input_audio_transcription.txt')#'Stored Data/Input/Transcription/input_audio_transcription.txt'
                with open(transcription_filename, 'a') as f:
                    f.write(alternate_text)

                return alternate_text

    except:
        print("Error! Check 'speech2Text' function")

recorded_audio_path = "C:\\Users\\krish\\OneDrive\\Desktop\\Study\\Gen AI\\Gen AI - MediaBot\\Data\\Input\\Input Audio\\"
transcripted_audio_path = 'C:\\Users\\krish\\OneDrive\\Desktop\\Study\\Gen AI\\Gen AI - MediaBot\\Data\\Output\\Audio Transcript\\'
speech2Text(recorded_audio_path,transcripted_audio_path)


latest_recording :  C:\Users\krish\OneDrive\Desktop\Study\Gen AI\Gen AI - MediaBot\Data\Input\Input Audio\recorded_audio.wav
latest_recording_filename :  recorded_audio.wav
Product table is unique product.


'Product table is unique product.'

In [14]:
recorded_audio_path = "C:\\Users\\krish\\OneDrive\\Desktop\\Study\\Gen AI\\Gen AI - MediaBot\\Data\\Input\\Input Audio\\recorded_audio.wav"
model = whisper.load_model("base")
audio = whisper.load_audio(recorded_audio_path)
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [15]:
_, probs = model.detect_language(mel)

In [16]:
k = sorted(probs.items(), key=lambda x:x[1])[-1]
k

('bn', 0.5023839473724365)

In [17]:
options = whisper.DecodingOptions(language= 'bn', fp16=False)

result = whisper.decode(model, mel, options)

In [19]:
result.text

'Product table is unique product to us.'